# Choosing a baseline model
In this section we discuss what is usually the first stage in tackling an imbalanced classfication problem, namely, to choose a good enough baseline model so further modelling developments are to be compared with this **baseline** model. The discussion can be guided by the following driving questions:
* What is the effect of class imbalance on different classification algorithms?
* Is class imbalance really a problem when there is enough data?
* What is the best benchmark model to start with? What does "best" actually mean in this situation?
* What can be done to mitigate the effect of imbalance on specific algorithms?

## Methodology
We intend to study the behavior (in terms of binary classification metrics) of different classification algorithms under various imbalance scenarios.
At this point we won't pay too much attention on optimizing the model's parameters. Rather, we will work with the vanilla versions of the models as we are interested in finding a "quick" generic approach to solving an imbalanced binary classification problem. 

The classical algorithm used for solving a binary classification problem is the Logistic Regression (citation needed!), two types of regularization (L1 and L2) will be considered to see the effect of regularization in the imbalanced case. On the other hand, it is also common to use tree-based ensemble methods to construct a model for binary classification. In this sense, we will also consider a couple of Tree ensemble models, such as Random Forests and Gradient Boosting Machines, such as `XGBoost` or `LGBM`.
